# Model Tutorial: MultiLayer Perceptron (Simple Neural Network)

The purpose of this notebook is to demonstrate how to train and predict a simple Neural Network used in this project. First, we will demonstrate the basic code, and then reproduce the results using a custom class `MLP` to make the code consistent for multiple models.

## Model Description

## Setup

In [1]:
import sys
sys.path.append('../src')
import pandas as pd
import numpy as np
import xgboost as xg
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
import yaml
# Local modules
from fmda_models import MLP
import reproducibility

## Read and Split Data

In [3]:
df = pd.read_pickle("../data/rocky_2023_05-09.pkl")
df = df.dropna(subset=['fm'])

In [4]:
# Set seed for reproducibility
reproducibility.set_seed(123)

# Create Data
X_train, X_test, y_train, y_test = train_test_split(df[["Ed", "Ew"]], df['fm'], test_size=.2)

resetting random seeds to 123


## Manually Code MLP

In [5]:
reproducibility.set_seed(123)

resetting random seeds to 123


In [6]:
params = {
    'hidden_units': 10,
    'activation': 'relu',
    'optimizer': 'adam',
    'epochs': 10,
    'batch_size': 32,
    'validation_split': 0.2,
    'dropout': 0.2,
    'learning_rate': 0.001  
}

In [7]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(params['hidden_units'], activation=params['activation'], input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dropout(params['dropout']),  # Dropout layer
    tf.keras.layers.Dense(1)  # Output layer with a single neuron for regression
])
optimizer = tf.keras.optimizers.Adam(learning_rate=params['learning_rate'])
model.compile(optimizer=optimizer, loss='mean_squared_error')

In [8]:
model.fit(X_train, y_train, epochs=params['epochs'], batch_size=params['batch_size'], validation_split=params['validation_split'])

Epoch 1/10
6741/6741 [==============================] - 33s 5ms/step - loss: 56.1067 - val_loss: 31.3427
Epoch 2/10
6741/6741 [==============================] - 23s 3ms/step - loss: 34.5822 - val_loss: 22.0599
Epoch 3/10
6741/6741 [==============================] - 20s 3ms/step - loss: 23.6894 - val_loss: 17.1313
Epoch 4/10
6741/6741 [==============================] - 19s 3ms/step - loss: 21.8261 - val_loss: 17.2398
Epoch 5/10
6741/6741 [==============================] - 20s 3ms/step - loss: 21.7568 - val_loss: 20.3542
Epoch 6/10
6741/6741 [==============================] - 20s 3ms/step - loss: 21.7191 - val_loss: 17.4936
Epoch 7/10
6741/6741 [==============================] - 21s 3ms/step - loss: 21.8298 - val_loss: 16.9511
Epoch 8/10
6741/6741 [==============================] - 22s 3ms/step - loss: 21.7062 - val_loss: 17.3332
Epoch 9/10
6741/6741 [==============================] - 20s 3ms/step - loss: 21.7534 - val_loss: 16.9663
Epoch 10/10
6741/6741 [==============================] 

In [9]:
fitted = model.predict(X_train)
preds = model.predict(X_test)

2107/2107 [==============================] - 4s 2ms/step


In [10]:
# Calculate RMSE for the training data
rmse_train = np.sqrt(mean_squared_error(y_train, fitted))

# Calculate R-squared for the training data
r2_train = r2_score(y_train, fitted)

# Calculate RMSE for the test data
rmse_test = np.sqrt(mean_squared_error(y_test, preds))

# Calculate R-squared for the test data
r2_test = r2_score(y_test, preds)

print("RMSE for training data:", rmse_train)
print("R-squared for training data:", r2_train)
print("RMSE for test data:", rmse_test)
print("R-squared for test data:", r2_test)

RMSE for training data: 4.179916290545039
R-squared for training data: 0.6022175936626519
RMSE for test data: 4.198117842503731
R-squared for test data: 0.60215609251223


## Reproduce using MLP Class

In [11]:
with open('params.yaml', 'r') as file:
    all_params = yaml.safe_load(file)

params = all_params["mlp"]
params["input_dim"] = X_train.shape[1] # Define Input shape based on X_train
params

{'hidden_units': 10,
 'activation': 'relu',
 'optimizer': 'adam',
 'epochs': 10,
 'batch_size': 32,
 'validation_split': 0.2,
 'dropout': 0.2,
 'learning_rate': 0.001,
 'input_dim': 2}

In [12]:
reproducibility.set_seed(123)

mlp = MLP(params)

resetting random seeds to 123


In [13]:
mlp.fit(X_train, y_train)

Epoch 1/10
6741/6741 [==============================] - 20s 3ms/step - loss: 56.1067 - accuracy: 0.0000e+00 - val_loss: 31.3427 - val_accuracy: 0.0000e+00
Epoch 2/10
6741/6741 [==============================] - 21s 3ms/step - loss: 34.5822 - accuracy: 0.0000e+00 - val_loss: 22.0599 - val_accuracy: 0.0000e+00
Epoch 3/10
6741/6741 [==============================] - 22s 3ms/step - loss: 23.6894 - accuracy: 0.0000e+00 - val_loss: 17.1313 - val_accuracy: 0.0000e+00
Epoch 4/10
6741/6741 [==============================] - 22s 3ms/step - loss: 21.8261 - accuracy: 0.0000e+00 - val_loss: 17.2398 - val_accuracy: 0.0000e+00
Epoch 5/10
5361/6741 [======================>.......] - ETA: 3s - loss: 21.7729 - accuracy: 0.0000e+00

KeyboardInterrupt: 

In [ ]:
fitted = mlp.predict(X_train)
preds = mlp.predict(X_test)

In [ ]:
# Calculate RMSE for the training data
rmse_train = np.sqrt(mean_squared_error(y_train, fitted))

# Calculate R-squared for the training data
r2_train = r2_score(y_train, fitted)

# Calculate RMSE for the test data
rmse_test = np.sqrt(mean_squared_error(y_test, preds))

# Calculate R-squared for the test data
r2_test = r2_score(y_test, preds)

print("RMSE for training data:", rmse_train)
print("R-squared for training data:", r2_train)
print("RMSE for test data:", rmse_test)
print("R-squared for test data:", r2_test)

In [ ]:
mlp.summary()

## Using Custom Loss

In [ ]:
def weighted_MSE(y_true,y_pred, val = -0.01):
    return K.mean(
        tf.multiply(
        tf.exp(tf.multiply(val, y_true)),
        tf.square(tf.subtract(y_pred, y_true))
    )
)